In [2]:
import pandas as pd

테스트용 DataFrame 생성

In [5]:
cust_dict = {
    'customer_name':['Alice', 'Tom', 'James', 'Yerin', 'Min', 'Katherine', 'John', 'Park', 'Maria', 'Derik', 'Jin'],
    'cust_country':['US', 'GB', 'US', 'KOR', 'KOR', 'GB', 'US', 'KOR', 'US', 'GB', 'KOR'],
    'gender':['F', 'M', 'M', 'F', 'M', 'F', 'M', 'M', 'F', 'M', 'F'],
    'grade':[3, 1, 2, 5, 4, 3, 5, 7, 2, 4, 5],
    'age':[25, 34, 26, 33, 67, 29, 54, 21, 77, 29, 16]
}
customer = pd.DataFrame(cust_dict)

customer.head(11)

,customer_name,cust_country,gender,grade,age
0,Alice,US,F,3,25
1,Tom,GB,M,1,34
2,James,US,M,2,26
3,Yerin,KOR,F,5,33
4,Min,KOR,M,4,67
5,Katherine,GB,F,3,29
6,John,US,M,5,54
7,Park,KOR,M,7,21
8,Maria,US,F,2,77
9,Derik,GB,M,4,29


## Pandas Groupby

groupby key 컬럼으로 DataFrameGroupby 생성

In [15]:
cust_group = customer.groupby('cust_country')
cust_group

cust_group은 groupby를 손쉽게 진행할 수 있는 하나의 object 객체이다. 해당 객체를 통해 다양한 agg 계산을 진행해볼 수 있다. 

In [18]:
cust_group.head()

,customer_name,cust_country,gender,grade,age
0,Alice,US,F,3,25
1,Tom,GB,M,1,34
2,James,US,M,2,26
3,Yerin,KOR,F,5,33
4,Min,KOR,M,4,67
5,Katherine,GB,F,3,29
6,John,US,M,5,54
7,Park,KOR,M,7,21
8,Maria,US,F,2,77
9,Derik,GB,M,4,29


cust_group을 출력하게 되면, 기존 DataFrame과 크게 차이가 없어 보인다. 하지만 새로운 DataFrame을 생성하고 groupby를 진행하게 되면 그 차이를 확인해볼 수 있다.

개별 aggregation 컬럼 및 aggregation 연산별로 API 호출

In [24]:
cust_agg = pd.DataFrame()
cust_agg

""


In [32]:
cust_agg['sum_grade'] = cust_group['grade'].sum()
cust_agg['max_grade'] = cust_group['grade'].max()
cust_agg.head()

,sum_grade,max_grade
cust_country,,
GB,8,4
KOR,21,7
US,12,5


cust_agg라는 빈 DataFrame을 생성한 후에, cust_group에서 groupby를 진행한 값을 컬럼별로 추가해주는 과정이다.

In [37]:
cust_agg['avg_age'] = cust_group['age'].mean()
cust_agg['max_age'] = cust_group['age'].max()
cust_agg.head()

,sum_grade,max_grade,avg_age,max_age
cust_country,,,,
GB,8,4,30.666667,34
KOR,21,7,34.250000,67
US,12,5,45.500000,77


위 이미지에서 주의할 점은 'cust_country' 즉, group by에서의 키 컬럼은 컬럼이 아닌 인덱스로 설정되어 있다.   
reset_index() 메서드를 통해 인덱스를 컬럼으로 변환한다. 

In [40]:
cust_agg = cust_agg.reset_index()
cust_agg.head()

,cust_country,sum_grade,max_grade,avg_age,max_age
0,GB,8,4,30.666667,34
1,KOR,21,7,34.250000,67
2,US,12,5,45.500000,77


agg() 함수의 인자로 리스트를 입력하는 경우에도각 컬럼별로 aggregation API가 적용된다.

In [42]:
cust_agg1 = cust_group['grade'].agg(['sum', 'max'])
cust_agg2 = cust_group['age'].agg(['mean', 'max'])
print(cust_agg1.head())
print(cust_agg2.head())

              sum  max
cust_country          
GB              8    4
KOR            21    7
US             12    5
                   mean  max
cust_country                
GB            30.666667   34
KOR           34.250000   67
US            45.500000   77


In [45]:
cust_agg = cust_agg1.merge(cust_agg2, on='cust_country', how='left')
cust_agg.head()

,sum,max_x,mean,max_y
cust_country,,,,
GB,8,4,30.666667,34
KOR,21,7,34.250000,67
US,12,5,45.500000,77


동일한 이름의 컬럼이 존재할 경우, max_x 또는 max_y 와 같이 컬럼이 구별되어 나오기는 하나, 각 컬럼이 의미하는 데이터를 확인하기 어렵다.  
aggreagation API를 적용할 때에 각 컬럼에 이름을 부여할 수 있다. 

In [50]:
cust_agg = pd.DataFrame()
cust_agg[['sum_grade', 'max_grade']] = cust_group['grade'].agg(['sum','max'])
cust_agg[['avg_age', 'max_age']] = cust_group['age'].agg(['mean','max'])
cust_agg.head()

,sum_grade,max_grade,avg_age,max_age
cust_country,,,,
GB,8,4,30.666667,34
KOR,21,7,34.250000,67
US,12,5,45.500000,77


agg()에 인자로 Aggregation 컬럼값과 Aggregation 함수를 리스트를 입력한 dictionary값을 입력하여 사용할 수 있다

In [53]:
agg_dict={
    'grade':['sum', 'max', 'min','count'],
    'age':['mean','max','min','count']
}

cust_agg = cust_group.agg(agg_dict)
cust_agg.head()

grade                      age              
               sum max min count       mean max min count
cust_country                                             
GB               8   4   1     3  30.666667  34  29     3
KOR             21   7   4     4  34.250000  67  16     4
US              12   5   2     4  45.500000  77  25     4

위와 같이 dictionary값을 Aggregatrion 함수 리스트에 전달할 경우, 멀티컬럼(multi-column)을 생성하게 된다.

In [58]:
print(cust_agg.columns)

MultiIndex([('grade',   'sum'),
            ('grade',   'max'),
            ('grade',   'min'),
            ('grade', 'count'),
            (  'age',  'mean'),
            (  'age',   'max'),
            (  'age',   'min'),
            (  'age', 'count')],
           )


In [60]:
[('-').join(column) for column in cust_agg.columns]

['grade-sum',
 'grade-max',
 'grade-min',
 'grade-count',
 'age-mean',
 'age-max',
 'age-min',
 'age-count']

join 메